In [1]:
%load_ext autoreload
%autoreload 2

from api_tricount import *


In [2]:

data, dict = fetch_data()
data = build_data(data)
detail = split_data(data, dict)
postes, result = concat_data(detail, dict)



    # --------------------------------- #
    # Connection to Google Sheet        #
    # and extract Spreadsheets          #
    #  - exported tricount data         #
    #  - category dictonnary            #
    # --------------------------------- #
    
 -- data fetched
(1023, 17)
Index(['Unnamed: 0', 'Titre', 'Montant', 'Devise', 'Taux de change',
       'Montant dans la devise du tricount (EUR)', 'Type de transaction',
       'Catégorie', 'Payé par', 'Payé par Commun', 'Payé par Lucie',
       'Payé par Vincent', 'Impacté à Commun', 'Impacté à Lucie',
       'Impacté à Vincent', 'Date & heure', 'URL des images'],
      dtype='object')
 -- dict fetched 
{'Alimentation': 'Quotidien', 'Shopping': 'Achats', 'Logement': 'Achats', 'Loyer & charges': 'Quotidien', 'Divertissement': 'Extra', 'Sport': 'Loisir', 'Restaurant & bar': 'Extra', 'Investissement': 'Investissement', 'Transport': 'Quotidien', 'Week-end': 'Loisir', 'Soin de santé': 'Quotidien', 'Livres': 'Achats', 'Activités': 'Lois

In [83]:
%load_ext autoreload
%autoreload 2
from api_portfolio import build_data


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
data = build_data()
s = data.iloc[-1]


    # ----------------------------------------------
    # Get data, concate, merge and extract metrics
    # for portfolio Dashboard 
    # ----------------------------------------------
    
              Market Currency     Forex IsDepot  PriceFmt AmountFmt  ValueFmt
Asset                                                                        
€           EUREUR=X        €  EUREUR=X    TRUE  {:.0f} €    {:.0f}  {:.0f} €
$              EUR=X        €  EUREUR=X    TRUE  {:.4f} €    {:.0f}  {:.0f} €
USDT           EUR=X        €  EUREUR=X    TRUE  {:.4f} €    {:.0f}  {:.0f} €
ESE           ESE.PA        €  EUREUR=X          {:.2f} €    {:.0f}  {:.2f} €
PUST                        €  EUREUR=X          {:.2f} €    {:.0f}  {:.2f} €
RS2K                        €  EUREUR=X          {:.2f} €    {:.0f}  {:.2f} €
OBLI         OBLI.PA        €  EUREUR=X          {:.2f} €    {:.0f}  {:.0f} €
GOLD         GOLD.PA        €  EUREUR=X          {:.2f} €    {:.0f}  {:.0f} €
BTC          BTC-USD     U

In [85]:
# s['PnL', 'ZEN', '$']
s[:, 'ZEN', '$'].index


Index(['Amount', 'Deposit', 'DepositEUR', 'Cotation', 'IsDepot', 'Currency',
       'PriceFmt', 'AmountFmt', 'ValueFmt'],
      dtype='object')

In [14]:
np.unique(assets['Currency'])

array(['$', 'USDT', '€'], dtype=object)

In [27]:
assets['USDT']['Market']

KeyError: 'USDT'

In [96]:
import streamlit as st
import gspread as gs 
import pandas as pd
import numpy as np
import yfinance as yf

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)


# gc = gs.service_account_from_dict(st.secrets['gcp_service_account'])
# ss = gc.open_by_key(st.secrets['portfolio'].spreadsheet_key)
# dicts = pd.DataFrame(ss.worksheet('Dict').get_all_records())
# operation = pd.DataFrame(ss.worksheet('Operations').get_all_records()).sort_values('Date').astype({'Date': 'datetime64[ns]'}).set_index('Date')
# greenbull = pd.DataFrame(ss.worksheet('GREENBULL').get_all_records()).sort_values('Date').astype({'Date': 'datetime64[ns]'}).set_index('Date')
# assets = dicts.set_index('Asset')
# print(assets, '\n')

# market = yf.download(' '.join([*assets['Forex'], *assets['Market'][:-1]]), start='2021-04-28')['Close']
# # print(market.columns)
# market = pd.concat([market, greenbull], axis=1)
# market = pd.concat([market], keys=['Market'], axis=1)
# market = pd.concat([market], keys=['Cotation'], axis=1)

operation: pd.DataFrame = operation
assets: pd.DataFrame = assets
market: pd.DataFrame = market

df = market.ffill().bfill()
# print(df.head())

for portfolio in np.unique(operation['Portfolio']):
    print(portfolio)
    dfp: pd.DataFrame = operation[operation['Portfolio'] == portfolio]

    print(f"\tAssets")
    for asset in np.unique(dfp['Asset']):
        print(f"\t\t{assets.loc[asset]['Class']}:\t{asset}")

        dfa: pd.DataFrame = dfp[dfp['Asset'] == asset]
        dfa = dfa.groupby('Date').agg({'Quantity': "sum", 'Total': "sum"}).reindex(df.index).fillna(0)

        df['Quantity', portfolio, asset] = dfa['Quantity']
        df['Total', portfolio, asset] = dfa['Total']
    
    print(f"\tCurrency")
    for currency in np.unique(dfp['Currency']):
        print(f"\t\t{assets.loc[currency]['Class']}:\t{currency}")

        dfc: pd.DataFrame = dfp[dfp['Currency'] == currency]
        dfc = dfc.groupby('Date').agg({'Quantity': "sum", 'Total': "sum"}).reindex(df.index).fillna(0)
        
        if ('Quantity', portfolio, currency) not in df.columns:
            df['Quantity', portfolio, currency] = 0
        if ('Total', portfolio, currency) not in df.columns:
            df['Total', portfolio, currency] = 0

        df['Quantity', portfolio, currency] -= dfc['Total']
        df['Total', portfolio, currency] -= dfc['Total'] * df['Cotation', 'Market', assets.loc[currency]['Market']]


    for asset in np.unique([*dfp['Asset'], *dfp['Currency']]):
        print(asset)

        amount, invest = pd.Series(dtype=float), pd.Series(dtype=float)
        for idx in df.index:
            amount[idx] = np.sum(df.loc[:idx]['Quantity', portfolio, asset])
            invest[idx] = np.sum(df.loc[:idx]['Total', portfolio, asset])

        df = pd.concat([df, pd.DataFrame({
                ('Amount', portfolio, asset): amount,
                ('Invested', portfolio, asset): invest,
            })], axis=1)

    for asset in np.unique(dfp['Asset']):
        toEUR = df['Cotation', 'Market', assets.loc[asset]['Forex']]
        df['Class', portfolio, asset] = assets.loc[asset]['Class']
        df['PriceFmt', portfolio, asset] = assets.loc[asset]['PriceFmt']
        df['ValueFmt', portfolio, asset] = assets.loc[asset]['ValueFmt']
        df['AmountFmt', portfolio, asset] = assets.loc[asset]['AmountFmt']

    #     df['Amount', portfolio, asset] = df['Quantity', portfolio, asset]
    #     df['Invested', portfolio, asset] = df['Total', portfolio, asset]
    #     df['Amount', portfolio, assets.loc[asset]['Currency']] -= df['Invested', portfolio, asset]
    #     df['InvestedEUR', portfolio, asset] = df['Invested', portfolio, asset] / toEUR
    #     df['Invested', portfolio, assets.loc[asset]['Currency']] -= df['InvestedEUR', portfolio, asset]

    # for asset in np.unique(dfp['Asset']):
    #     df['PRU', portfolio, asset] = df['Invested', portfolio, asset] / df['Amount', portfolio, asset]
    #     df['Value', portfolio, asset] = df['Amount', portfolio, asset] * df['Cotation', portfolio, asset]
    #     df['PnL', portfolio, asset] = df['Value', portfolio, asset] - df['Invested', portfolio, asset]
    #     df['ValueEUR', portfolio, asset] = df['Value', portfolio, asset] / toEUR
    #     df['PnLEUR', portfolio, asset] = df['PnL', portfolio, asset] / toEUR


# dates = ["2021-04-30", "2021-04-30", "2021-07-30", "2021-07-30", "2021-08-10", "2021-08-10", "2021-10-01", "2021-10-01", "2021-11-01", "2021-11-01", "2021-12-01", "2021-12-01", "2021-12-31", "2022-02-01", "2022-02-01", "2022-03-24", "2022-03-24", "2022-04-01", "2022-04-01", "2022-04-01", "2022-04-05", "2022-04-05", "2022-04-13", "2022-04-13", "2022-05-02", "2022-05-02", "2022-04-19", "2022-04-19", "2022-06-01", "2022-06-01", "2022-06-01", "2022-06-01", "2022-07-01", "2022-07-04", "2022-07-05", "2022-07-05", "2022-08-05", "2022-08-05", "2022-08-25", "2022-10-12", "2022-10-12", "2022-10-12", "2023-02-10"]
dates = ["2021-04-30","2021-07-30","2021-08-10","2021-10-01","2021-11-01","2021-12-01","2021-12-31","2022-02-01","2022-03-24","2022-04-01","2022-04-05","2022-04-13","2022-05-02","2022-04-19","2022-06-01","2022-07-01","2022-07-04","2022-07-05","2022-08-05","2022-08-25","2022-10-12","2023-02-10"]
# df.loc[dates]
# df.head()
df.loc[dates, pd.IndexSlice[['Quantity', 'Amount', 'Total', 'Invested'], 'ZEN', ['BTC', 'USDT', 'EUR']]]


              Market Currency     Forex    Class  PriceFmt AmountFmt  ValueFmt
Asset                                                                         
EUR         EUREUR=X                     Deposit                              
€           EUREUR=X      EUR  EUREUR=X     Cash  {:.0f} €    {:.0f}  {:.0f} €
$              EUR=X      EUR  EUREUR=X     Cash  {:.4f} €    {:.0f}  {:.0f} €
USDT           EUR=X     EUR   EUREUR=X     Cash  {:.4f} €    {:.0f}  {:.0f} €
ESE           ESE.PA        €  EUREUR=X   Stocks  {:.2f} €    {:.0f}  {:.2f} €
PUST                        €  EUREUR=X   Stocks  {:.2f} €    {:.0f}  {:.2f} €
RS2K                        €  EUREUR=X   Stocks  {:.2f} €    {:.0f}  {:.2f} €
OBLI         OBLI.PA        €  EUREUR=X    Bonds  {:.2f} €    {:.0f}  {:.0f} €
GOLD         GOLD.PA        €  EUREUR=X    Commo  {:.2f} €    {:.0f}  {:.0f} €
BTC          BTC-USD     USDT  EURUSD=X   Crypto  {:.0f} $    {:.4f}  {:.0f} $
ETH                      USDT  EURUSD=X   Crypto  {:

Quantity                    Amount                     Total  \
                ZEN                       ZEN                       ZEN   
                BTC     USDT     EUR      BTC     USDT      EUR     BTC   
2021-04-30  0.00000     0.00  -500.0  0.00000     0.00   -500.0    0.00   
2021-07-30  0.00000     0.00  -500.0  0.00000     0.00  -1000.0    0.00   
2021-08-10  0.00000     0.00  -300.0  0.00000     0.00  -1300.0    0.00   
2021-10-01  0.00000     0.00  -500.0  0.00000     0.00  -1800.0    0.00   
2021-11-01  0.00000     0.00  -500.0  0.00000     0.00  -2300.0    0.00   
2021-12-01  0.00000     0.00 -1000.0  0.00000     0.00  -3300.0    0.00   
2021-12-31  0.00000     0.00     0.0  0.00000     0.00  -3300.0    0.00   
2022-02-01  0.00000     0.00  -500.0  0.00000     0.00  -3800.0    0.00   
2022-03-24  0.00000     0.00 -1200.0  0.00000     0.00  -5000.0    0.00   
2022-04-01  0.00000  2200.00 -2500.0  0.00000  2200.00  -7500.0    0.00   
2022-04-05  0.00480  -220.00     0.0  0.00480  1980.00  -7500.0  220.00   
2022-04-13  0.00540  -220.00     0.0  0.01020  1760.00  -7500.0  220.00   
2022-05-02  0.00560  -220.00     0.0  0.01580  1540.00  -9500.0  220.00   
2022-04-19  0.00000     0.00 -2000.0  0.01020  1760.00  -9500.0    0.00   
2022-06-01  0.00700  -220.00  -500.0  0.02280  1320.00 -10000.0  220.00   
2022-07-01  0.01144  -220.00     0.0  0.03424  1100.00 -10000.0  220.00   
2022-07-04  0.00000     0.00     0.0  0.03424  1100.00 -10000.0    0.00   
2022-07-05  0.00000     0.00 -1500.0  0.03424  1100.00 -11500.0    0.00   
2022-08-05  0.00950  -220.00     0.0  0.04374   880.00 -11500.0  220.00   
2022-08-25  0.00000     0.00  -500.0  0.04374   880.00 -12000.0    0.00   
2022-10-12  0.01150  -219.84     0.0  0.05524   660.16 -12000.0  219.84   
2023-02-10  0.00000     0.00     0.0  0.05524   660.16 -12000.0    0.00   

                                Invested                        
                                     ZEN                        
                   USDT     EUR      BTC         USDT      EUR  
2021-04-30     0.000000  -500.0     0.00     0.000000   -500.0  
2021-07-30     0.000000  -500.0     0.00     0.000000  -1000.0  
2021-08-10     0.000000  -300.0     0.00     0.000000  -1300.0  
2021-10-01     0.000000  -500.0     0.00     0.000000  -1800.0  
2021-11-01     0.000000  -500.0     0.00     0.000000  -2300.0  
2021-12-01     0.000000 -1000.0     0.00     0.000000  -3300.0  
2021-12-31     0.000000     0.0     0.00     0.000000  -3300.0  
2022-02-01     0.000000  -500.0     0.00     0.000000  -3800.0  
2022-03-24     0.000000 -1200.0     0.00     0.000000  -5000.0  
2022-04-01  2000.000000 -2500.0     0.00  2000.000000  -7500.0  
2022-04-05  -200.439804     0.0   220.00  1799.560196  -7500.0  
2022-04-13  -203.211805     0.0   440.00  1596.348392  -7500.0  
2022-05-02  -208.808600     0.0   660.00  1387.539792  -9500.0  
2022-04-19     0.000000 -2000.0   440.00  1596.348392  -9500.0  
2022-06-01  -204.954197  -500.0   880.00  1182.585595 -10000.0  
2022-07-01  -209.970204     0.0  1100.00   972.615391 -10000.0  
2022-07-04     0.000000     0.0  1100.00   972.615391 -10000.0  
2022-07-05     0.000000 -1500.0  1100.00   972.615391 -11500.0  
2022-08-05  -214.680401     0.0  1320.00   757.934990 -11500.0  
2022-08-25     0.000000  -500.0  1320.00   757.934990 -12000.0  
2022-10-12  -226.635258     0.0  1539.84   531.299732 -12000.0  
2023-02-10     0.000000     0.0  1539.84   531.299732 -12000.0

In [37]:
('Ctation', 'Market',   'BTC-USD') in df.columns

False